# INTRO
Along with the presentation, the notebook also describes the project goals, data, methods, and results. It must include at least three visualizations which correspond to three business recommendations.

The graded elements for the Jupyter Notebook are:
Business Understanding,
Data Understanding,
Data Preparation,
Data Analysis,
Visualization, and
Code Quality

Include takeaways for visuals in markdown

In [1]:
#NOTE: I ADDED IMPORT RE AT THE END

# import packages

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from random import gauss, seed
import sklearn.metrics as metrics
import statsmodels.api as sm
import sqlite3
import re

%matplotlib inline

In [2]:
# read all csv and tsv data files into dataframes

df1=pd.read_csv("zippedData/tn.movie_budgets.csv.gz")
##df2=pd.read_csv("zippedData/tmdb.movies.csv.gz")
##df3=pd.read_csv("zippedData/rt.reviews.tsv.gz", sep="\t", encoding='windows-1252')
##df4=pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep="\t", encoding='windows-1252')
df5=pd.read_csv("zippedData/bom.movie_gross.csv.gz")

In [3]:
#establish sql connection and read in the im.db
conn = sqlite3.connect("zippedData/im.db/im.db")

## Data preparation and cleaning

In [4]:
#NOTE: I MADE A CHANGE IN THE QUERY

df_sql_merged = pd.read_sql("""
SELECT per.primary_name, bas.primary_title, bas.genres, bas.movie_id, bas.start_year
FROM persons as per
    JOIN principals as pri
    ON pri.person_id = per.person_id
    JOIN movie_basics as bas
    ON pri.movie_id = bas.movie_id
WHERE category = "producer"
""", conn)

In [5]:
# this code converts the release date column to datetime
df1['release_date'] = pd.to_datetime(df1['release_date'])
df1['year'] = df1['release_date'].apply(lambda x: x.year)
df1['month'] = df1['release_date'].apply(lambda x: x.month)

In [6]:
# NOTE: THIS IS A NEW CELL
# create a key from the title and year

df_sql_merged['title_year'] = df_sql_merged['primary_title'] + df_sql_merged['start_year'].astype('str')
df1['title_year'] = df1['movie'] + df1['year'].astype('str')
df5['title_year'] = df5['title'] + df5['year'].astype('str')

In [7]:
# NOTE: I COMBINED TWO CELLS HERE

# create regex pattern and replacement dictionary
change_hyphen = re.compile('\s*-\s+')

replace_dict = {'â\x80\x99': "'",
                'â\x80\x94': ' - ',
                'â\x80\x93': ' - ',
                'â\x80¦': '…',
                'Ã\xad': 'í',
                'Ä\x81': 'ā',
                'Ã¤': 'ä',
                'Ã¥': 'å',
                'Ã¡': 'á',
                'Ã§': 'ç',
                'Ã©': 'é',
                'Ã¨': 'è',
                'Ã«': 'ë',
                'Ãª': 'ê',
                'à¬': 'ì',
                'Ä«': 'ī',
                'Ã´': 'ô',
                'Ã³': 'ó',
                'Ã¸': 'ø',
                'Ã¹': 'ù',
                'Ã¼': 'ü',
                'Ã»': 'ū',
                'Ã': 'à',    # This has to be after the two-digit patterns
                'Â': '',     # I think this is a non-breaking space
                change_hyphen: ': '
               }

In [8]:
# this identifies the correct pattern, and replaces all particular bads with goods

df1['movie'] = df1['movie'].replace(replace_dict, regex=True)
df_sql_merged['primary_title'] = df_sql_merged['primary_title'].replace(replace_dict, regex=True)

In [9]:
# NOTE: I COMBINED THIS CELL WITH ANOTHER, GOT RID OF THE FUNCTION, NO LONGER INCLUDE WORLDWIDE_GROSS

# this code removes $ and , from currency columns.  It takes a Series names as an argument and removes
# those two characters from each value, then recasts the value as int64

replace_chars = '\$|,'
columns_to_clean = ['production_budget', 'domestic_gross']
df1[columns_to_clean] = df1[columns_to_clean].apply(lambda x: x.str.replace(replace_chars,'').astype('int64'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   object        
 6   year               5782 non-null   int64         
 7   month              5782 non-null   int64         
 8   title_year         5782 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 406.7+ KB


In [10]:
# NOTE: I MADE CHANGES IN THIS CELL: ADDED RETURN COL, MERGE ON  NEW TITLE_YEAR KEY, DROPPING DOMESTIC_GROSS==0

df_csv_merged = pd.merge(df1, df5, on='title_year', how='left')
df_csv_merged = df_csv_merged[df_csv_merged['domestic_gross_x'] != 0]
df_csv_merged['return'] = (df_csv_merged['domestic_gross_x'] / df_csv_merged['production_budget']) - 1
df_merged = pd.merge(df_csv_merged, df_sql_merged, on="title_year", how = "left")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6729 entries, 0 to 6728
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 6729 non-null   int64         
 1   release_date       6729 non-null   datetime64[ns]
 2   movie              6729 non-null   object        
 3   production_budget  6729 non-null   int64         
 4   domestic_gross_x   6729 non-null   int64         
 5   worldwide_gross    6729 non-null   object        
 6   year_x             6729 non-null   int64         
 7   month              6729 non-null   int64         
 8   title_year         6729 non-null   object        
 9   title              2353 non-null   object        
 10  studio             2352 non-null   object        
 11  domestic_gross_y   2353 non-null   float64       
 12  foreign_gross      2054 non-null   object        
 13  year_y             2353 non-null   float64       
 14  return  

In [11]:
# NOTE: I CHANGED THIS CELL TO NOW DROP WORLDWIDE_GROSS
df_merged = df_merged.drop(columns=["domestic_gross_y", "worldwide_gross", "foreign_gross", "year_y", "id", "start_year"])

In [12]:
# NOTE: I ADDED ANOTHER COLUMN TO THE RENAME LINE
# Clean column names and filter for films in the 2010-2018 time period

df_merged.rename(columns={'year_x': 'year', 'domestic_gross_x': 'domestic_gross'}, inplace=True)
earliest_date = pd.to_datetime('2010-01-01')
latest_date = pd.to_datetime('2018-12-31')
df_merged = df_merged.loc[(df_merged['release_date']>=earliest_date) & (df_merged['release_date']<=latest_date)]

,release_date,movie,production_budget,domestic_gross,year,month,title_year,title,studio,return,primary_name,primary_title,genres,movie_id
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,2011,5,Pirates of the Caribbean: On Stranger Tides2011,Pirates of the Caribbean: On Stranger Tides,BV,-0.412899,NaN,NaN,NaN,NaN
3,2015-05-01,Avengers: Age of Ultron,330600000,459005868,2015,5,Avengers: Age of Ultron2015,Avengers: Age of Ultron,BV,0.388403,Kevin Feige,Avengers: Age of Ultron,"Action,Adventure,Sci-Fi",tt2395427
4,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,2017,12,Star Wars Ep. VIII: The Last Jedi2017,NaN,NaN,0.956408,NaN,NaN,NaN,NaN
5,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2015,12,Star Wars Ep. VII: The Force Awakens2015,NaN,NaN,2.060988,NaN,NaN,NaN,NaN
6,2018-04-27,Avengers: Infinity War,300000000,678815482,2018,4,Avengers: Infinity War2018,Avengers: Infinity War,BV,1.262718,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6700,2016-03-18,Krisha,30000,144822,2016,3,Krisha2016,NaN,NaN,3.827400,NaN,NaN,NaN,NaN
6703,2010-10-15,Down Terrace,30000,9812,2010,10,Down Terrace2010,NaN,NaN,-0.672933,NaN,NaN,NaN,NaN
6706,2017-01-27,Emily,27000,3547,2017,1,Emily2017,NaN,NaN,-0.868630,NaN,NaN,NaN,NaN
6712,2010-04-02,Breaking Upwards,15000,115592,2010,4,Breaking Upwards2010,NaN,NaN,6.706133,NaN,NaN,NaN,NaN
